<a href="https://colab.research.google.com/github/acts701/LLM/blob/main/250102_%EA%B0%9C%EB%B0%9C%EC%9D%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문장 교정 시도
- 새해가 된다고 잘못된 STT가 저절로 좋아지지 않는다.
- clova, google 등에서 유료로 STT 된 파일을 활용하여 품질이 다소 떨어진 상태로 STT 된 파일을 교정하고자 시도
```python
    raw_sentences = list(kss.split_sentences(raw_text))
    corrected_sentences = []
    
    for raw_sentence in raw_sentences:
        # 참조 텍스트와 비교하여 가장 유사한 문장 찾기
        match = process.extractOne(raw_sentence, reference_text, scorer=fuzz.token_sort_ratio)
        if match and match[1] > 80:  # 유사도가 80% 이상이면 교정
            corrected_sentences.append(match[0])
        else:
            corrected_sentences.append(raw_sentence)  # 유사도가 낮으면 원문 유지
```
- 시도는 좋았지만 결과는 전/후가 거의 비슷함.

# 학습
- 모델을 바꿔봐도 결과는 비슷함
``` python
from sentence_transformers import SentenceTransformer, models, losses
import math
import logging
pretrained_model_name = 'snunlp/KR-BERT-char16424'
embedding_model = models.Transformer(pretrained_model_name, max_seq_length=256)
pooling_model = models.Pooling(embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)

model = SentenceTransformer(modules=[embedding_model, pooling_model])
train_loss = losses.MultipleNegativesRankingLoss(model)
```
- text 품질도 높여야 하지만 LLM 이론 공부가 부족한건 아닌지도 봐야겠다.